In [1]:
#Bruker ResNet arkitektur kombinert med PyTorch 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

c:\Users\47472\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [4]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [5]:
print(bhArray.shape)
print(sphArray.shape)

(15000, 50, 50, 3)
(15000, 50, 50, 3)


In [6]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [8]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [9]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [10]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [12]:
#ResidualBlock block to be used in ResNet model
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):

        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [56]:
class ResNetModel(nn.Module):
    def __init__(self, block, layers, num_classes=2):

        super(ResNetModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        
        self.inplanes = 64
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)

        # self.avgPool = nn.AvgPool2d(7, stride=1)

        self.fc0 = nn.Linear(2048, num_classes)

    def forward(self, x:Tensor):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)

        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        # x = self.avgPool(x)
        x = torch.flatten(x, 1)
        x = self.fc0(x)

        return x


    #https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)



In [57]:
from torchvision import models
from torchsummary import summary

model = ResNetModel(ResidualBlock, [3, 4, 6, 3]).to(device)
summary(model, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 25, 25]           9,472
            Conv2d-2           [-1, 64, 12, 12]          36,928
       BatchNorm2d-3           [-1, 64, 12, 12]             128
              ReLU-4           [-1, 64, 12, 12]               0
            Conv2d-5           [-1, 64, 12, 12]          36,928
       BatchNorm2d-6           [-1, 64, 12, 12]             128
              ReLU-7           [-1, 64, 12, 12]               0
     ResidualBlock-8           [-1, 64, 12, 12]               0
            Conv2d-9           [-1, 64, 12, 12]          36,928
      BatchNorm2d-10           [-1, 64, 12, 12]             128
             ReLU-11           [-1, 64, 12, 12]               0
           Conv2d-12           [-1, 64, 12, 12]          36,928
      BatchNorm2d-13           [-1, 64, 12, 12]             128
             ReLU-14           [-1, 64,

In [27]:
def trainFunction(config=None):

    #init wandb
    with wandb.init(project="PyTorch", name="resnetmodel", config=config):
        config = wandb.config

        #initialize variables
        resNetModel = ResNetModel(ResidualBlock, [3, 4, 6, 3]).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(resNetModel.parameters(), lr=config.learning_rate)
        #initialize variables end


        #data
        trainLoader = DataLoader(train, shuffle=True, batch_size=config.batch_size)
        testLoader = DataLoader(test, shuffle=True, batch_size=config.batch_size)
        #data end


        wandb.watch(resNetModel, criterion, log='all')

        
        #START
        for epoch in range(50):

            #training variables
            trainRunningLoss = 0.0
            correct = 0
            y_pred = []
            y_true = []
            #training loop
            resNetModel.train()
            for i, data in enumerate(trainLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                output = resNetModel(inputs.permute(0,3,1,2))
                output1 = (torch.max(torch.exp(output.to(device)), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = criterion(output, labels.type(torch.LongTensor).to(device))
                loss.backward()
                optimizer.step()
                
                trainRunningLoss += loss.item()
            #training loop end
            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            trainAccuracy = correct / len(y_true)
            #training variables end

            #test variables
            testRunningLoss = 0.0
            y_pred = []
            y_true = []
            #test loop
            resNetModel.eval()
            for j, data in enumerate(testLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                output = resNetModel(inputs.permute(0,3,1,2))# Feed Network

                output1 = (torch.max(torch.exp(output.to(device)), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = criterion(output, labels.type(torch.LongTensor).to(device))
                testRunningLoss += loss.item()
            #test loop end

            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            testAccuracy = correct / len(y_true)
            epochLoss = trainRunningLoss/len(bhArray)
            #test variables end

            #wandb log
            wandb.log({"Train epoch_loss":epochLoss, "Test epoch_loss": testRunningLoss, "Train accuracy": trainAccuracy,"Test accuracy": testAccuracy})

        #END


In [58]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'batch_size': {'values': [50]},
        'learning_rate': {'values': [0.01]}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="test")


Create sweep with ID: pe4pf87e
Sweep URL: https://wandb.ai/g13hvl2023/test/sweeps/pe4pf87e


In [59]:
wandb.agent(sweep_id, function=trainFunction, count=10)

wandb: Agent Starting Run: lzpi0byg with config:
wandb: 	batch_size: 50
wandb: 	learning_rate: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
